#  Weather Forecasting - Défi IA - Kaggle Competition

Preprocessing from Raphael Sourty

In [42]:
import os
import pandas as pd
import numpy as np

import lightgbm as lgbm
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

### Load and clean train set:

I chose to rename some columns of the data. These data is a good example of what to avoid. In Python we should always name our columns in lower case and separate the words with an underscore. The variable "station" replaces "number_sta". I'm deliberately getting rid of the date. We won't need it, it is not defined in the test data. The day is now an integer from 0 to 729 (729 for the last day of the second year). I also created a test column that will distinguish test data from training data.

In [3]:
train = pd.read_csv(
    'data/Y_train.csv', delimiter=',', 
    parse_dates=['date'], 
    infer_datetime_format=True,
    dtype = {"number_sta": int, "Ground_truth": float},
).rename(
    columns = {"Ground_truth": "y", "Id": "id", "number_sta": "station"}
).sort_values(["date", "station"]).assign(test=False)

train["month"] = train.date.dt.month.astype(int)
train["day"] = train["id"].str.split("_").str[1].astype(int)
train = train.drop("date", axis = "columns")

In [4]:
train.head()

,station,y,id,test,month,day
0,14066001,3.4,14066001_0,False,1,0
1,14126001,0.5,14126001_0,False,1,0
2,14137001,3.4,14137001_0,False,1,0
3,14216001,4.0,14216001_0,False,1,0
4,14296001,13.3,14296001_0,False,1,0


### Load and clean test:

I apply the same logic to the test data, my goal being to gather in the same dataframe the test data and the training data in order to calculate the features more easily.

In [8]:
test = pd.read_csv(
   'data/Baseline_observation_test.csv',
    dtype = {"Prediction": float}
).assign(Prediction=None, test=True).rename(columns = {"Prediction": "y", "Id": "id", "number_sta": "station"})

test["day"] = test["id"].str.split("_").str[1].astype(int)

test = pd.merge(
    left = test,
    right = pd.read_csv('data/Id_month_test.csv').rename(columns = {"day_index": "day"}),
    on = ["day"],
    how = "left",
)

test["station"] = test["id"].str.split("_").str[0].astype(int)

In [9]:
test.head()

,id,y,test,day,month,station
0,14066001_149,None,True,149,1,14066001
1,14126001_149,None,True,149,1,14126001
2,14137001_149,None,True,149,1,14137001
3,14216001_149,None,True,149,1,14216001
4,14296001_149,None,True,149,1,14296001


#### Concat training set and test set:

The training and test data are formatted in the same way, so we can concatenate them. The df dataframe will now be the main dataset. We will add the features to this datafarame `df`. Features are new columns to help us estimate the target variable, we create these features manually. I also choose to delete the `id` column because I can easily find it from the station variable and the day variable.

In [10]:
df = pd.concat([train, test], axis = "rows")
df = df.drop("id", axis = "columns")
df.head()

,station,y,test,month,day
0,14066001,3.4,False,1,0
1,14126001,0.5,False,1,0
2,14137001,3.4,False,1,0
3,14216001,4.0,False,1,0
4,14296001,13.3,False,1,0


#### Station latitude, longitude, height

I will add to the df dataframe `df` the latitude and longitude data set.

In [12]:
stations = pd.read_csv(
    'data/stations_coordinates.csv',
    dtype = {"number_sta": int}         
).rename(
    columns={"number_sta": "station", "lat": "latitude", "lon": "longitude", "height_sta": "height"}
)

In [13]:
stations.head()

,station,latitude,longitude,height
0,86118001,46.477,0.985,120.0
1,86149001,46.917,0.025,60.0
2,56081003,48.050,-3.660,165.0
3,53215001,47.790,-0.710,63.0
4,22135001,48.550,-3.380,148.0


In [14]:
df = pd.merge(
    left = df,
    right = stations,
    on = "station",
    how = "left",
)

In [15]:
df.head()

,station,y,test,month,day,latitude,longitude,height
0,14066001,3.4,False,1,0,49.334,-0.431,2.0
1,14126001,0.5,False,1,0,49.145,0.042,125.0
2,14137001,3.4,False,1,0,49.180,-0.456,67.0
3,14216001,4.0,False,1,0,48.928,-0.149,155.0
4,14296001,13.3,False,1,0,48.795,-1.037,336.0


#### Loading the training features:

The files `data/Test/X_station_test.csv` and `data/Train/X_station_train.csv` contain the available weather stations records with a fine granularity (record per station and per hour). We will use this data to enrich the `df` dataframe that we have previously created. We will have to aggregates those features to add them to our dataframe `df`.

First I will start by loading these data and cleaning them.

In [19]:
train_features = pd.read_csv(
    'data/X_station_train.csv',
    parse_dates=['date'], 
    infer_datetime_format=True,
    dtype = {"number_sta": int},
).rename(
    columns = {"number_sta": "station", "Id": "id"}
).assign(test=False)

train_features["month"] = train_features["date"].dt.month.astype(int)
train_features["day"] = train_features["date"].dt.day.astype(int)
train_features["hour"] = train_features["date"].dt.hour.astype(int)

train_features = train_features.drop(["date", "id"], axis = "columns")

In [21]:
train_features.head()

,station,ff,t,td,hu,dd,precip,test,month,day,hour
0,14066001,3.05,279.28,277.97,91.4,200.0,0.0,False,1,1,0
1,14066001,2.57,278.76,277.45,91.4,190.0,0.0,False,1,1,1
2,14066001,2.26,278.27,277.02,91.7,181.0,0.0,False,1,1,2
3,14066001,2.62,277.98,276.95,93.0,159.0,0.0,False,1,1,3
4,14066001,2.99,277.32,276.72,95.9,171.0,0.0,False,1,1,4


#### Loading the test features:

I apply the same logic to the test features to be able to concatenate all the test and train features. We can then build aggregates and join them to the `df` dataframe.

In [22]:
test_features = pd.read_csv(
    'data/X_station_test.csv', 
).rename(
    columns = {"Id": "id"}
).assign(test=True)

In [24]:
test_features.head()

,dd,hu,td,t,ff,precip,month,id,test
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4,True
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5,True
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6,True
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7,True
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8,True


In [25]:
test_features["station"] = test_features["id"].str.split("_").str[0].astype(int)
test_features["day"] = test_features["id"].str.split("_").str[1].ffill().astype(int)
test_features["hour"] = test_features["id"].str.split("_").str[2].ffill().astype(int)

In [26]:
test_features = test_features.drop(["id"], axis = "columns")

#### Features

I now create the `features` dataframe. This dataframe will contain all the records useful for creating features for the `df` data set.

In [27]:
features = pd.concat([train_features, test_features], axis = "rows")

In [28]:
features.head()

,station,ff,t,td,hu,dd,precip,test,month,day,hour
0,14066001,3.05,279.28,277.97,91.4,200.0,0.0,False,1,1,0
1,14066001,2.57,278.76,277.45,91.4,190.0,0.0,False,1,1,1
2,14066001,2.26,278.27,277.02,91.7,181.0,0.0,False,1,1,2
3,14066001,2.62,277.98,276.95,93.0,159.0,0.0,False,1,1,3
4,14066001,2.99,277.32,276.72,95.9,171.0,0.0,False,1,1,4


#### New Features

Let's start create new features since we have our two clean dataframes `df` and `features`. Our main objective will be to create aggregates of the `features` dataframe and then perform left-join to the `df` dataframe.

In [29]:
def make_agg(x, on, by, how):
    """Function to make aggregates easilly.
    
    Parameters
    ----------
        x (pd.DataFrame): Features dataframe.
        on (list): List of targets aggregates.
        by (list): List of columns we will use to group data.
        how (list): List of statistics.
    
    """
    columns = []
    for col in on:
        for stat in how:
            columns.append(f"{stat}_{col}_by_{'_'.join(by)}")
    x = x.groupby(by)[on].agg(how)  
    x.columns = columns
    return x
    

For example, here is a set of features that might be useful. We compute the mean, sum and standard deviation of variables `["ff", "t", "td", "hu", "dd", "precip"]` grouped by `["station", "day"]`.

In [30]:
make_agg(
    x = features, 
    on = ["ff", "t", "td", "hu", "dd", "precip"], 
    by = ["station", "day"], 
    how = ["mean", "sum", "std"]
)

mean_ff_by_station_day  sum_ff_by_station_day  \
station  day                                                  
14047002 31                      NaN                   0.00   
         39                      NaN                   0.00   
         100                     NaN                   0.00   
         223                     NaN                   0.00   
         236                     NaN                   0.00   
...                              ...                    ...   
95690001 358                9.727083                 233.45   
         359                1.460000                  35.04   
         360                5.399167                 129.58   
         361                1.925833                  46.22   
         362                4.542083                 109.01   

              std_ff_by_station_day  mean_t_by_station_day  \
station  day                                                 
14047002 31                     NaN             281.276250   
         39                     NaN             283.776250   
         100                    NaN             282.565000   
         223                    NaN             285.415000   
         236                    NaN             283.064167   
...                             ...                    ...   
95690001 358               1.332854             288.737083   
         359               0.907639             293.641667   
         360               1.318830             292.657917   
         361               0.922732             289.524583   
         362               1.652721             275.627917   

              sum_t_by_station_day  std_t_by_station_day  \
station  day                                               
14047002 31                6750.63              1.174193   
         39                6810.63              1.232110   
         100               6781.56              0.407057   
         223               6849.96              0.713576   
         236               6793.54              0.540442   
...                            ...                   ...   
95690001 358               6929.69              2.124037   
         359               7047.40              3.054662   
         360               7023.79              3.622908   
         361               6948.59              2.679572   
         362               6615.07              2.433932   

              mean_td_by_station_day  sum_td_by_station_day  \
station  day                                                  
14047002 31                      NaN                   0.00   
         39                      NaN                   0.00   
         100                     NaN                   0.00   
         223                     NaN                   0.00   
         236                     NaN                   0.00   
...                              ...                    ...   
95690001 358              285.856250                6860.55   
         359              289.546667                6949.12   
         360              286.341667                6872.20   
         361              288.077083                6913.85   
         362              271.831667                6523.96   

              std_td_by_station_day  mean_hu_by_station_day  \
station  day                                                  
14047002 31                     NaN                     NaN   
         39                     NaN                     NaN   
         100                    NaN                     NaN   
         223                    NaN                     NaN   
         236                    NaN                     NaN   
...                             ...                     ...   
95690001 358               1.809506               83.204167   
         359               0.824878               78.387500   
         360               0.624135               68.950000   
         361               1.934633               91.416667   
         362               3.3

We just have to join these new features with the `pd.merge` method:

In [31]:
agg = make_agg(
    x = features, 
    on = ["ff", "t", "td", "hu", "dd", "precip"], 
    by = ["station", "day"], 
    how = ["mean", "sum", "std"]
)


df = pd.merge(
    left = df,
    right = agg,
    on = ["station", "day"],
    how = "left",
)

#### Train test split:

We separate the test game from the training game here to proceed with the model training and inference.

In [32]:
# The "test" column is a boolean column which is equal to True for test and False for train.
X_train = df[df["test"] == False].copy()
X_test = df[df["test"]].copy()

Some of the samples in the target variable are missing, we will drop it for now but it could be interesting to retrieve them:

In [33]:
X_train = X_train[X_train["y"].notnull()]

# y_train is the target variable to train our model.
y_train = X_train["y"].astype(float).copy()

We re-create the id for the submission and set it as the index of X_test

In [34]:
X_test["id"] = X_test["station"].astype(str) + "_" + X_test["day"].astype(str)
X_test = X_test.set_index("id")

We will need to drop some columns to avoid errors such as station, test and day and y. 

In [35]:
columns_to_drop = ["station", "y", "test", "day"]

X_train = X_train.drop(columns_to_drop, axis = "columns")
X_test = X_test.drop(columns_to_drop, axis = "columns")

In [36]:
X_train.head()

,month,latitude,longitude,height,mean_ff_by_station_day,sum_ff_by_station_day,std_ff_by_station_day,mean_t_by_station_day,sum_t_by_station_day,std_t_by_station_day,...,std_td_by_station_day,mean_hu_by_station_day,sum_hu_by_station_day,std_hu_by_station_day,mean_dd_by_station_day,sum_dd_by_station_day,std_dd_by_station_day,mean_precip_by_station_day,sum_precip_by_station_day,std_precip_by_station_day
0,1,49.334,-0.431,2.0,3.370000,80.88,0.695764,279.474583,6707.39,2.634342,...,1.149634,89.125000,2139.0,11.261062,206.750000,4962.0,18.666990,0.008333,0.2,0.040825
1,1,49.145,0.042,125.0,NaN,0.00,NaN,279.164583,6699.95,2.375394,...,1.288028,93.520833,2244.5,8.066947,NaN,0.0,NaN,0.020833,0.5,0.102062
2,1,49.180,-0.456,67.0,3.388333,81.32,0.462767,279.266667,6702.40,2.535209,...,1.303820,89.908333,2157.8,8.780954,197.708333,4745.0,26.278573,0.008333,0.2,0.040825
3,1,48.928,-0.149,155.0,2.695000,64.68,0.474763,279.235417,6701.65,2.134809,...,0.742995,86.620833,2078.9,9.849696,213.583333,5126.0,26.806743,0.000000,0.0,0.000000
4,1,48.795,-1.037,336.0,NaN,0.00,NaN,277.874167,6668.98,0.908343,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.000000,0.0,0.000000


In [37]:
X_test.head()

,month,latitude,longitude,height,mean_ff_by_station_day,sum_ff_by_station_day,std_ff_by_station_day,mean_t_by_station_day,sum_t_by_station_day,std_t_by_station_day,...,std_td_by_station_day,mean_hu_by_station_day,sum_hu_by_station_day,std_hu_by_station_day,mean_dd_by_station_day,sum_dd_by_station_day,std_dd_by_station_day,mean_precip_by_station_day,sum_precip_by_station_day,std_precip_by_station_day
id,,,,,,,,,,,,,,,,,,,,,
14066001_149,1,49.334,-0.431,2.0,5.237917,125.71,1.805901,282.099583,6770.39,1.415856,...,1.100198,81.720833,1961.3,5.016753,224.625000,5391.0,23.384894,0.058333,1.4,0.285774
14126001_149,1,49.145,0.042,125.0,NaN,0.00,NaN,280.895833,6741.50,1.114033,...,1.359493,92.291667,2215.0,3.697580,NaN,0.0,NaN,0.041667,1.0,0.204124
14137001_149,1,49.180,-0.456,67.0,5.547083,133.13,1.853623,281.167917,6748.03,1.710721,...,1.403252,83.266667,1998.4,4.955775,218.458333,5243.0,26.131738,0.016667,0.4,0.081650
14216001_149,1,48.928,-0.149,155.0,2.667083,64.01,1.020667,280.804583,6739.31,1.552857,...,1.477365,83.345833,2000.3,6.885270,207.375000,4977.0,32.714593,0.033333,0.8,0.127404
14296001_149,1,48.795,-1.037,336.0,NaN,0.00,NaN,279.570417,6709.69,1.042090,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.150000,3.6,0.547723


#### Handle missing values:

I choose to replace the missing values by -1, there are surely better solutions to replace the missing values.

# Model:

#### Cross-validation

I choose here to use the LightGbm model. It is a very powerful model that wins most of the machine learning competitions. It is interesting because we can use it to measure the weight of features in the decision making of the model. Moreover, the Lightgbm API is quite practical. It is important to look for the best model parameters.

In [43]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [48]:
param_grid = {
    "boosting_type": ['gbdt','dart'],
    "learning_rate": [0.001, 0.01, 0.1, 0.2],
    "reg_alpha": [0.001, 0.01, 0.1],
    "reg_lambda": [0.001, 0.01, 0.1]
}

cross_val = model_selection.KFold(5, shuffle=True, random_state=42)

lgbm_model = lgbm.LGBMRegressor(
    n_estimators=100, # Number of trees
    random_state=42,
)

grid_search_model = GridSearchCV(lgbm_model, param_grid, cv=5)

### Fitting

In [50]:
sub = pd.Series(0.0, index=X_test.index)
error_validation = 0
k_folds = 5 

for fit_idx, val_idx in cross_val.split(X_train, y_train):

    x_fit = X_train.iloc[fit_idx]
    x_val = X_train.iloc[val_idx]
    
    y_fit = y_train.iloc[fit_idx] 
    y_val = y_train.iloc[val_idx]

    fitted_model = grid_search_model.fit(x_fit, y_fit)

    # Computation of the loss
    print("Best Score = %f, Best Parameters  = %s" %
      (1.-fitted_model.best_score_, fitted_model.best_params_))

    # Validation predictions:
    error_validation += mean_absolute_percentage_error(
        y_true = fitted_model.predict(x_val) + 1, 
        y_pred = y_val + 1,
    ) / k_folds
    
    # Bagging
    # Since we train 5 models using cross-validation with k=5 folds, we average the prediction of each models.
    # It helps a lot!
    sub += fitted_model.predict(X_test) / k_folds
    
# To avoid zero-division errors, METEO FRANCE added 1 to the target variable, so we do it too.
sub += 1

Best Score = 1.030821, Best Parameters  = {'boosting_type': 'gbdt', 'learning_rate': 0.001, 'reg_alpha': 0.1, 'reg_lambda': 0.1}
Best Score = 1.030406, Best Parameters  = {'boosting_type': 'gbdt', 'learning_rate': 0.001, 'reg_alpha': 0.1, 'reg_lambda': 0.001}
Best Score = 1.026357, Best Parameters  = {'boosting_type': 'gbdt', 'learning_rate': 0.001, 'reg_alpha': 0.001, 'reg_lambda': 0.001}
Best Score = 1.032484, Best Parameters  = {'boosting_type': 'gbdt', 'learning_rate': 0.001, 'reg_alpha': 0.1, 'reg_lambda': 0.01}
Best Score = 1.033003, Best Parameters  = {'boosting_type': 'gbdt', 'learning_rate': 0.001, 'reg_alpha': 0.1, 'reg_lambda': 0.1}


#### Validation MAPE

In [51]:
print(f"MAPE on validation set: {error_validation}")

MAPE on validation set: 86.04398167358573


## Our submission:

In [55]:
sub

id
14066001_149    3.075437
14126001_149    3.053722
14137001_149    3.075437
14216001_149    3.044619
14296001_149    3.132800
                  ...   
86137003_293    2.815710
86165005_293    2.815710
86273001_293    2.815710
91200002_293    2.815710
95690001_293    2.815710
Length: 85140, dtype: float64

In [56]:
sub = sub.rename("Prediction").reset_index().rename(columns = {"id": "Id"})

In [57]:
sub.to_csv("submission.csv", index=False)
sub

,Id,Prediction
0,14066001_149,3.075437
1,14126001_149,3.053722
2,14137001_149,3.075437
3,14216001_149,3.044619
4,14296001_149,3.132800
...,...,...
85135,86137003_293,2.815710
85136,86165005_293,2.815710
85137,86273001_293,2.815710
85138,91200002_293,2.815710


#### Useful resources:

https://maxhalford.github.io/blog/target-encoding/

https://explained.ai/gradient-boosting/